# Sélection et jointure de données

In [118]:
import sqlite3

In [119]:
import pandas as pd

df = pd.read_csv('./datasets/persons_cleaned.csv')
df

,first_name,last_name,national_number,birthdate,age,gender,age_category,company,full_name
0,Édouard,Hoarau,57.04.25-655.94,1957-04-25,66,Male,47-113 Senior,Huet s a s,Édouard Hoarau
1,Nath,Raymond,66.07.21-762.08,1966-07-21,57,Female,47-113 Senior,Gimenez,Nath Raymond
2,Élise,Reynaud,80.07.22-299.56,1980-07-22,43,Male,43-46 À point,Thibault boyer s a s,Élise Reynaud
3,Laure,Gérard,89.04.01-944.45,1989-04-01,34,Female,0-38 Jeune,Weiss,Laure Gérard
4,Célina,Ledoux,58.05.11-442.46,1958-05-11,65,Female,47-113 Senior,Blanc,Célina Ledoux
...,...,...,...,...,...,...,...,...,...
995,Arthur,Clerc,61.08.18-911.28,1961-08-18,62,Male,47-113 Senior,Boutin sa,Arthur Clerc
996,Aimée,Collin,47.05.05-995.41,1947-05-05,76,Male,47-113 Senior,Dias peron sa,Aimée Collin
997,Olivie,Bazin,89.02.08-520.65,1989-02-08,35,Female,0-38 Jeune,Dijoux s a r l,Olivie Bazin
998,Patricia,Lucas,82.10.28-722.89,1982-10-28,41,Female,39-42 Adulte,Marechal,Patricia Lucas


In [120]:
haircuts = pd.read_csv('./datasets/persons_haircut.csv')
haircuts

,NATIONAL_NUMBER_CAPS_LOVER,haircut,white_hair_count
0,72032216395,Tonsure,276
1,58030793744,Chignon,457
2,50040785273,Tonsure,NaN
3,66110473107,Chignon,145
4,59030152487,Hi-top fade,60
...,...,...,...
716,67012996622,Dreadlocks,470
717,47052146973,mULLET,478
718,82060234807,Chignon,159
719,83052589205,Dreadlocks,424


In [121]:
# quick cleaning 
haircuts = haircuts.dropna()

# rename column NATIONAL_NUMBER_CAPS_LOVER to national_number
haircuts = haircuts.rename(columns={'NATIONAL_NUMBER_CAPS_LOVER': 'national_number'})

In [81]:
df = df.merge(haircuts, on='national_number', how='left')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [85]:
# remove non numeric values from the column national_number
df['national_number'] = df['national_number'].str.replace(r'\D', '', regex=True)

df = df.merge(haircuts, on='national_number', how='left')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [122]:
df['national_number'] = df['national_number'].str.replace(r'\D', '', regex=True)
df['national_number'] = df['national_number'].astype(int)

df = df.merge(haircuts, on='national_number', how='left')
df

,first_name,last_name,national_number,birthdate,age,gender,age_category,company,full_name,haircut,white_hair_count
0,Édouard,Hoarau,57042565594,1957-04-25,66,Male,47-113 Senior,Huet s a s,Édouard Hoarau,Chignon,458
1,Nath,Raymond,66072176208,1966-07-21,57,Female,47-113 Senior,Gimenez,Nath Raymond,NaN,NaN
2,Élise,Reynaud,80072229956,1980-07-22,43,Male,43-46 À point,Thibault boyer s a s,Élise Reynaud,Pixie cut,119
3,Laure,Gérard,89040194445,1989-04-01,34,Female,0-38 Jeune,Weiss,Laure Gérard,Chignon,298
4,Célina,Ledoux,58051144246,1958-05-11,65,Female,47-113 Senior,Blanc,Célina Ledoux,Bouffant,35
...,...,...,...,...,...,...,...,...,...,...,...
995,Arthur,Clerc,61081891128,1961-08-18,62,Male,47-113 Senior,Boutin sa,Arthur Clerc,Dreadlocks,345
996,Aimée,Collin,47050599541,1947-05-05,76,Male,47-113 Senior,Dias peron sa,Aimée Collin,Butch cut,492
997,Olivie,Bazin,89020852065,1989-02-08,35,Female,0-38 Jeune,Dijoux s a r l,Olivie Bazin,Pompadour,155
998,Patricia,Lucas,82102872289,1982-10-28,41,Female,39-42 Adulte,Marechal,Patricia Lucas,NaN,NaN


In [123]:
con = sqlite3.connect('persons.db')
df.to_sql('persons', con, if_exists='replace', index=False)
haircuts.to_sql('haircuts', con, if_exists='replace', index=False)

638

In [124]:
# drop first_name, last_name, national_number
df = df.drop(columns=['first_name', 'last_name', 'national_number'])


In [91]:
df.sample(10)

,birthdate,age,gender,age_category,company,full_name,haircut,white_hair_count
148,1973-01-02,51,Male,47-113 Senior,Rolland aubert s a r l,Denise Baron,NaN,NaN
846,1948-10-29,75,Male,47-113 Senior,Collet,Corinne Schmitt,Undercut,87k
539,1966-08-28,57,Male,47-113 Senior,Roger,Maurice Paris,Butch cut,353
838,1997-11-21,26,Female,0-38 Jeune,Bouvet,Noémi Dupont,Layered hair,394
616,1950-02-17,74,Female,47-113 Senior,Arnaud,Laetitia Laroche,Bouffant,456k
116,1992-07-30,31,Female,0-38 Jeune,Auger s a r l,Thibault Pires,NaN,NaN
292,1995-09-28,28,Male,0-38 Jeune,Lecomte sa,Martin Fontaine,NaN,NaN
820,1979-03-10,44,Male,43-46 À point,Maillet s a r l,Aimé Lefebvre,Hi-top fade,56
343,1958-05-07,65,Female,47-113 Senior,Perez vaillant sarl,Victor Ferrand,NaN,NaN
561,1992-04-03,31,Male,0-38 Jeune,Lefevre s a s,Odette Simon,devilock,375


In [92]:
# haircuts nan -> 'Bald'
df['haircut'] = df['haircut'].fillna('Bald')

# replace k in white_hair_count with 1000
df['white_hair_count'] = df['white_hair_count'].str.replace('k', '000')
df['white_hair_count'] = df['white_hair_count'].fillna(0)
df['white_hair_count'] = df['white_hair_count'].astype(int)


In [93]:
df

,birthdate,age,gender,age_category,company,full_name,haircut,white_hair_count
0,1957-04-25,66,Male,47-113 Senior,Huet s a s,Édouard Hoarau,Chignon,458
1,1966-07-21,57,Female,47-113 Senior,Gimenez,Nath Raymond,Bald,0
2,1980-07-22,43,Male,43-46 À point,Thibault boyer s a s,Élise Reynaud,Pixie cut,119
3,1989-04-01,34,Female,0-38 Jeune,Weiss,Laure Gérard,Chignon,298
4,1958-05-11,65,Female,47-113 Senior,Blanc,Célina Ledoux,Bouffant,35
...,...,...,...,...,...,...,...,...
995,1961-08-18,62,Male,47-113 Senior,Boutin sa,Arthur Clerc,Dreadlocks,345
996,1947-05-05,76,Male,47-113 Senior,Dias peron sa,Aimée Collin,Butch cut,492
997,1989-02-08,35,Female,0-38 Jeune,Dijoux s a r l,Olivie Bazin,Pompadour,155
998,1982-10-28,41,Female,39-42 Adulte,Marechal,Patricia Lucas,Bald,0


In [96]:
import plotly.express as px

# plot sum of persons with by haircut
fig = px.bar(df.groupby('haircut').size().reset_index(name='count'), x='haircut', y='count')
fig.show()


In [108]:
# remove Bald persons
df = df[df['haircut'] != 'Bald']

# plot sum of persons with by haircut
fig = px.bar(df.groupby('haircut').size())
fig.show()

In [109]:
import plotly.express as px

# plot sum of white_hair_count by haircut
fig = px.bar(
        df.groupby('haircut').agg({'white_hair_count': 'mean'}).reset_index(), 
        x='haircut', 
        y='white_hair_count')
fig.show()


In [126]:

df = pd.read_sql('SELECT * FROM persons LEFT JOIN haircuts ON persons.national_number = haircuts.national_number', con)
df


,first_name,last_name,national_number,birthdate,age,gender,age_category,company,full_name,haircut,white_hair_count,national_number,haircut,white_hair_count
0,Édouard,Hoarau,57042565594,1957-04-25,66,Male,47-113 Senior,Huet s a s,Édouard Hoarau,Chignon,458,5.704257e+10,Chignon,458
1,Nath,Raymond,66072176208,1966-07-21,57,Female,47-113 Senior,Gimenez,Nath Raymond,None,None,NaN,None,None
2,Élise,Reynaud,80072229956,1980-07-22,43,Male,43-46 À point,Thibault boyer s a s,Élise Reynaud,Pixie cut,119,8.007223e+10,Pixie cut,119
3,Laure,Gérard,89040194445,1989-04-01,34,Female,0-38 Jeune,Weiss,Laure Gérard,Chignon,298,8.904019e+10,Chignon,298
4,Célina,Ledoux,58051144246,1958-05-11,65,Female,47-113 Senior,Blanc,Célina Ledoux,Bouffant,35,5.805114e+10,Bouffant,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Arthur,Clerc,61081891128,1961-08-18,62,Male,47-113 Senior,Boutin sa,Arthur Clerc,Dreadlocks,345,6.108189e+10,Dreadlocks,345
996,Aimée,Collin,47050599541,1947-05-05,76,Male,47-113 Senior,Dias peron sa,Aimée Collin,Butch cut,492,4.705060e+10,Butch cut,492
997,Olivie,Bazin,89020852065,1989-02-08,35,Female,0-38 Jeune,Dijoux s a r l,Olivie Bazin,Pompadour,155,8.902085e+10,Pompadour,155
998,Patricia,Lucas,82102872289,1982-10-28,41,Female,39-42 Adulte,Marechal,Patricia Lucas,None,None,NaN,None,None
